# Module Project - Gender Classificaton

## Before you start

You'll need the latest version of the **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.13.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: marshmallow, azure-storage-blob, pyyaml, azure-storage-file-datalake, azure-core, isodate, azure-mgmt-core, azure-common, jsonschema, msrest, pyjwt, strictyaml, colorama, opencensus-ext-azure, typing-extensions, azure-storage-file-share, tqdm, pydash
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Task 1 : Command Job with Custom Tracking

## Custom tracking with MLflow

When running a script as a job you can use MLflow in your training script to track the model. MLflow allows you to track any custom parameters, metrics, or artifacts you want to store with your job output.

Run the following cells to create the **train-model-mlflow.py** script in the **src** folder. The script trains a classification model by using the **diabetes.csv** file in the same folder, which is passed as an argument. 

Review the code below to find that the script will import `mlflow` and log:

- The regularization rate as a **parameter**. 
- The accuracy and AUC as **metrics**.
- The plotted ROC curve as an **artifact**.

In [4]:
import os

# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


In [5]:
%%writefile $script_folder/train-model-mlflow.py
# import libraries
import mlflow
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

def main(args):
    # read data
    df = get_data(args.training_data)

    # Normalize numerical data
    norm_df = normalize_data(df)

    # Encode categorical data
    encod_df = encode_data(norm_df)

    # Split data
    X_train, X_test, y_train, y_test = split_data(encod_df)

    # Train model
    model = train_model(args.reg_rate, X_train, X_test, y_train, y_test)

    # Evaluate model
    eval_model(model, X_test, y_test)

# function that reads the data
def get_data(path):
    print("Reading data...")
    df = pd.read_csv(path)
    
    return df

# Function to split the data
def split_data(df):
    print("Splitting data...")
    X = df[["long_hair","forehead_width_cm","forehead_height_cm","nose_wide","nose_long","lips_thin","distance_nose_to_lip_long"]].values
    y = df['gender'].values
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
    return X_train, X_test, y_train, y_test

# Function to normalize numerical data
def normalize_data(df):
    print("Normalizing data...")
    numerical_columns = ["forehead_width_cm", "forehead_height_cm", "distance_nose_to_lip_long"]
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    return df

# Function to label encode categorical data
def encode_data(norm_df):
    print("Encoding data...")
    categorical_columns = ["long_hair", "nose_wide", "nose_long", "lips_thin", "gender"]
    encoder = LabelEncoder()
    for col in categorical_columns:
       norm_df[col] = encoder.fit_transform(norm_df[col])
    print("Encoded DataFrame:")
    print(norm_df.head())
    return norm_df   

# Function to train the model
def train_model(reg_rate, X_train, X_test, y_train, y_test):
    print("Training model...")
    mlflow.log_param("Regularization rate", reg_rate)
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)
    return model

# Function to evaluate the model
def eval_model(model, X_test, y_test):
    print("Evaluating model...")
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    mlflow.log_metric("Accuracy", acc)
    print('Accuracy:', acc)

    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test, y_scores[:, 1])
    mlflow.log_metric("AUC", auc)
    print('AUC:', auc)

    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:, 1])
    fig = plt.figure(figsize=(6, 4))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png")
    mlflow.log_artifact("ROC-Curve.png")
    

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--reg_rate", dest='reg_rate',
                        type=float, default=0.01)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Overwriting src/train-model-mlflow.py


Now, you can submit the script as a command job.

Run the cell below to train the model. 

In [6]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-mlflow.py --training_data gender_classification_v7_2__2_.csv",   #your dataset name within the folder
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="ismayilsiyad1",#change to your computr cluster name
    display_name="gender-train-mlflow",
    experiment_name="gender-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Monitor your job at https://ml.azure.com/runs/honest_drawer_gd5c4tkn12?wsid=/subscriptions/7784b486-c1df-4244-be68-cefd0413ea59/resourcegroups/rg-dp100-labs/workspaces/teseewfsd&tid=0c4563c3-3bc8-450f-8008-bc0edc7c121e


In the Studio, navigate to the **diabetes-train-mlflow** job to explore the overview of the command job you ran:

- Find the logged parameters in the **Overview** tab, under **Params**.
- Find the logged metrics in the **Metrics** tab.
- Find the logged artifacts in the **Images** tab (specifically for images), and in the **Outputs + logs** tab (all files).

## Task 2: Command Job with Auto Tracking

## Autologging with MLflow

Instead of using custom logging, MLflow can also automatically log any parameters, metrics, and artifacts. Autologging with MLflow requires only one line of code.

Run the following cell to create the **train-model-autolog.py** script in the **src** folder. The script trains a classification model by using the **diabetes.csv** file in the same folder, which is passed as an argument. 

Review the code below to find that the script will import `mlflow` and enables autologging with the line: 

`mlflow.autolog()`

In [7]:
%%writefile $script_folder/train-model-autolog.py
# import libraries
import mlflow
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

def main(args):
    # enable autologging
    mlflow.autolog()
    # read data
    df = get_data(args.training_data)

    # Normalize numerical data
    norm_df = normalize_data(df)

    # Encode categorical data
    encod_df = encode_data(norm_df)

    # Split data
    X_train, X_test, y_train, y_test = split_data(encod_df)

    # Train model
    model = train_model(args.reg_rate, X_train, X_test, y_train, y_test)

    # Evaluate model
    eval_model(model, X_test, y_test)

# function that reads the data
def get_data(path):
    print("Reading data...")
    df = pd.read_csv(path)
    
    return df

# Function to split the data
def split_data(df):
    print("Splitting data...")
    X = df[["long_hair","forehead_width_cm","forehead_height_cm","nose_wide","nose_long","lips_thin","distance_nose_to_lip_long"]].values
    y = df['gender'].values
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
    return X_train, X_test, y_train, y_test

# Function to normalize numerical data
def normalize_data(df):
    print("Normalizing data...")
    numerical_columns = ["forehead_width_cm", "forehead_height_cm", "distance_nose_to_lip_long"]
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    return df

# Function to label encode categorical data
def encode_data(norm_df):
    print("Encoding data...")
    categorical_columns = ["long_hair", "nose_wide", "nose_long", "lips_thin", "gender"]
    encoder = LabelEncoder()
    for col in categorical_columns:
       norm_df[col] = encoder.fit_transform(norm_df[col])
    print("Encoded DataFrame:")
    print(norm_df.head())
    return norm_df   

# Function to train the model
def train_model(reg_rate, X_train, X_test, y_train, y_test):
    print("Training model...")
    mlflow.log_param("Regularization rate", reg_rate)
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)
    return model

# Function to evaluate the model
def eval_model(model, X_test, y_test):
    print("Evaluating model...")
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)

    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test, y_scores[:, 1])
    print('AUC:', auc)

    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:, 1])
    fig = plt.figure(figsize=(6, 4))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png")
    
    

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--reg_rate", dest='reg_rate',
                        type=float, default=0.01)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Overwriting src/train-model-autolog.py


Now, you can submit the script as a command job.

Run the cell below to train the model. 

In [8]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data gender_classification_v7_2__2_.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="ismayilsiyad1",
    display_name="gender-train-autolog",
    experiment_name="gender-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/eager_crayon_jjzkdjhtl5?wsid=/subscriptions/7784b486-c1df-4244-be68-cefd0413ea59/resourcegroups/rg-dp100-labs/workspaces/teseewfsd&tid=0c4563c3-3bc8-450f-8008-bc0edc7c121e


## Task 3: Sweep Job for Hyperparameter Optimization

## Create the training script
Hyperparameter tuning is ideal when you want to train a machine learning models but vary the input parameters. You'll need to create a training script that expects an input parameter representing one of the algorithm's hyperparameters.

Run the following cells to create the **src** folder and the training script.

Note that the training script expects two input parameters:

- `--training_data` which expects a string. You'll specify the path to a registered data asset as the input training data.
- `--reg_rate` which expects a number, but has a default value of `0.01`. You'll use this input parameter for hyperparameter tuning.

In [9]:
%%writefile $script_folder/train.py
# import libraries
import mlflow
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

def main(args):
    # read data
    df = get_data(args.training_data)

    # Normalize numerical data
    norm_df = normalize_data(df)

    # Encode categorical data
    encod_df = encode_data(norm_df)

    # Split data
    X_train, X_test, y_train, y_test = split_data(encod_df)

    # Train model
    model = train_model(args.reg_rate, X_train, X_test, y_train, y_test)

    # Evaluate model
    eval_model(model, X_test, y_test)

# function that reads the data
def get_data(path):
    print("Reading data...")
    df = pd.read_csv(path)
    
    return df

# Function to split the data
def split_data(df):
    print("Splitting data...")
    X = df[["long_hair","forehead_width_cm","forehead_height_cm","nose_wide","nose_long","lips_thin","distance_nose_to_lip_long"]].values
    y = df['gender'].values
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
    return X_train, X_test, y_train, y_test

# Function to normalize numerical data
def normalize_data(df):
    print("Normalizing data...")
    numerical_columns = ["forehead_width_cm", "forehead_height_cm", "distance_nose_to_lip_long"]
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    return df

# Function to label encode categorical data
def encode_data(norm_df):
    print("Encoding data...")
    categorical_columns = ["long_hair", "nose_wide", "nose_long", "lips_thin", "gender"]
    encoder = LabelEncoder()
    for col in categorical_columns:
       norm_df[col] = encoder.fit_transform(norm_df[col])
    print("Encoded DataFrame:")
    print(norm_df.head())
    return norm_df   

# Function to train the model
def train_model(reg_rate, X_train, X_test, y_train, y_test):
    print("Training model...")
    mlflow.log_param("Regularization rate", reg_rate)
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)
    return model

# Function to evaluate the model
def eval_model(model, X_test, y_test):
    print("Evaluating model...")
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    mlflow.log_metric("Accuracy", acc)
    print('Accuracy:', acc)

    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test, y_scores[:, 1])
    mlflow.log_metric("AUC", auc)
    print('AUC:', auc)

    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:, 1])
    fig = plt.figure(figsize=(6, 4))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png")
    mlflow.log_artifact("ROC-Curve.png")
    

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--reg_rate", dest='reg_rate',
                        type=float, default=0.01)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Overwriting src/train.py


## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [10]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.gender_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "gender_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:genders-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="ismayilsiyad1",
    display_name="gender-train-mlflow",
    experiment_name="gender-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/epic_bell_9gb16gjrrb?wsid=/subscriptions/7784b486-c1df-4244-be68-cefd0413ea59/resourcegroups/rg-dp100-labs/workspaces/teseewfsd&tid=0c4563c3-3bc8-450f-8008-bc0edc7c121e


## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [11]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [12]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="ismayilsiyad1",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-gender"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=2, max_concurrent_trials=1, timeout=7200)

Run the following cell to submit the sweep job.

In [13]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/orange_dress_rv62rc6l6c?wsid=/subscriptions/7784b486-c1df-4244-be68-cefd0413ea59/resourcegroups/rg-dp100-labs/workspaces/teseewfsd&tid=0c4563c3-3bc8-450f-8008-bc0edc7c121e


Check the output of Sweep Job, and verify which hyper parameter value (Regularization) is giving the best accuracy

Now Let's check AutoML

## Task 4: Configure AutoML Job

## Prepare data

You don't need to create a training script for automated machine learning, but you do need to load the training data. 

In this case, you'll use a dataset containing details of diabetes patients. 

To pass a dataset as an input to an automated machine learning job, the data must be in tabular form and include a target column. For the data to be interpreted as a tabular dataset, the input dataset must be a **MLTable**.

A MLTable data asset has already been created for you during set-up. You can explore the data asset by navigating to the **Data** page. You'll retrieve the data asset here by specifying its name `diabetes-training-table` and version `1`. 

In [19]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:gender_table:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [20]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="ccluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="gender",
    primary_metric="accuracy",
    n_cross_validations=2,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=30, 
    trial_timeout_minutes=20, 
    max_trials=4,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [21]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/lime_yam_906gtgj0k2?wsid=/subscriptions/7784b486-c1df-4244-be68-cefd0413ea59/resourcegroups/rg-dp100-labs/workspaces/teseewfsd&tid=0c4563c3-3bc8-450f-8008-bc0edc7c121e
